# ```tenacity``` Tutorial #

### What is ```tenacity```? ###
```tenacity``` is an Apache 2.0 general-purpose retrying library that simplifies the task of adding retry behavior to just about anything. 

### Instalation ###
```tenacity``` can be installed via ```pip3 install tenacity```.

### Main Objects ###
**Functions**
- ```tenacity.retry()```: wrap a function with a new ```Retrying``` object;

**Classes**
- ```tenacity.Retrying(sleep: Sleep, stop: Stop, wait: Wait, retry: Retry, before: before_nothing, after: after_nothing, before_sleep: None, reraise: False, retry_error_cls: tenacity.RetryError, retry_error_callback: None)```: retrying controller sync class;
- ```tenacity.AsyncRetrying()```: retrying controller async class;

**Main Keyword Arguments**
- ```stop```
    - ```stop.stop_after_attempt(max_attempt_number: int)```: stops when the previous attempt >= ```max_attempt_number```;
    - ```stop.stop_after_delay(max_delay: float)```: stops when the time from the first attempt >= ```max_delay```;
    - ```stop.stop_when_event_set(event: threading.Event)```: stops when parsed ```threading``` event is set;
    - ```stop.stop_all(*stops)```: stops if all the ```stop``` conditions are valid;
    - ```stop.stop_any(*stops)```: stops if any the ```stop``` conditions are valid;
    <br><br>
- ```wait``` 
    - ```wait.wait_exponential(multiplier: float=1, max: float=4.6e+18, exp_base: float=2, min: float=0)```: strategy that applies exponential backoff, suitable for balancing retries against latency (formula: ```multiplier * exp_base**attempt```, bounded by ```min``` and ```max```);
    - ```wait.wait_exponential_jitter(initial: float=1, max: float=4.6e+18, exp_base: float=2, jitter: float=1)```: strategy that applies exponential backoff plus jitter (formula: ```initial * 2**attempt + random.uniform(0,jitter)```, bounded by ```min``` and ```max```);
    - ```wait.wait_fixed(wait: float)```: strategy that waits a fixed amount of time between each retry;
    - ```wait.wait.wait_incrementing(start: float=0, increment: float=100, max: float= 4.6e+18)```: strategy that waits an incremental amount of time after each attempt (starting with ```start``` and incrementing by ```increment``` on each attempt);
    - ```wait.wait_none```: strategy that doesn’t wait before retrying;
    - ```wait.wait_random(min: float=0, max: float=1)```: strategy that waits a random amount of time (bounded by ```min``` and ```max```);
    - ```wait.wait_random_exponential(multiplier: float=1, max: float=4.6e+18, exp_base: float=2, min: float=0)```: strategy that waits exponential random time in a geometrically expanding interval (formula: ```multiplier * exp_base**attempt```, bounded by ```min``` and ```max```);
    - ```wait.wait_chain(*strategies)```: chains 2+ waiting strategies (if all strategies are exhausted, last one is used thereafter);
    - ```wait.wait_combine(*strategies)```: combines several waiting strategies;
    <br><br>
- ```retry```
    - ```retry.retry_base```: abstract base class for retry strategies;
    - ```retry.retry_if_exception(predicate: bool])```: retries if an exception verifies a predicate;
    - ```retry.retry_if_exception_cause_type(exception_types: tuple:Exception_Types)```: retries if raised exception is of type ```Exception_Types```;
    - ```retry.retry_if_exception_message(message: str=None, match: str=None)```: retries if an exception message equals or matches;
    - ```retry.retry_if_not_exception_message(message: str=None, match: str=None)```: retries until an exception message equals or matches;
    - ```retry.retry_if_exception_type(exception_types: tuple:Exception_Types)```: retries if raised exception is of type ```Exception_Types```;
    - ```retry.retry_if_not_exception_type(exception_types: tuple:Exception_Types)```: retries if raised exception is not of type ```Exception_Types```;
    - ```retry.retry_if_result(predicate: bool)```: retries if the result verifies a predicate;
    - ```retry.retry_if_not_result(predicate: bool)```: retries if the result refutes a predicate;
    - ```retry.retry_unless_exception_type(exception_types: tuple:Exception_Types)```: retries until an exception is raised of type ```Exception_Types```;
    - ```retry.retry_all(*retries)```: retries if all the ```retries``` condition are valid;
    - ```retry.retry_any(*retries)```: retries if any of the ```retries``` condition are valid;
    <br><br>
- ```sleep```
    - ```nap.sleep(seconds: float)```: sleeps by ```seconds```;
    - ```nap.sleep_using_event(event: threading.Event)```: sleeps until an event is set;
    <br><br>
- ```before```
    - ```before.before_nothing(retry_state: RetryState)```: strategy that does nothing before next retry;
    - ```before.before_log(logger: Logger, log_level: int)```: strategy that logs the attempt to ```Logger``` before next retry;
    <br><br>
- ```before_sleep```
    - ```before_sleep.before_sleep_nothing(retry_state: RetryState)```: strategy that does nothing before sleep;
    - ```before_sleep.before_sleep_log(logger: Logger, log_level: int)```: strategy that logs the attempt to ```Logger``` before sleep;
    <br><br>
- ```after```
    - ```after.after_nothing(retry_state: RetryState)```: strategy that does nothing after a retry failed;
    - ```after.after_log(logger: Logger, log_level: int, sec_format: str='%0.3f')```: strategy that logs the attempt to ```Logger``` after a retry failed;

**Other Keyword Arguments** 
- ```reraise```: when ```True```, sends the original exception (triggered by the code) to the end of the stack trace (instead of returning the default ```RetryError```);
- ```retry_error_cls```: the class that shoud be returned when a retry takes place (default: ```RetryError```);
- ```retry_error_callback```: receives the custom callback function to be called after all retries failed (more below);

### Custom Callbacks ###
We can define custom callback functions to be called after all retries fail (withou raising an exception). The callback should accept 1 parameter called ```retry_state``` that contains all information about the current retry invocation.

It’s also possible to define custom callbacks for most of the keyword arguments (above). Those are already defined in ```tenacity``` and (as stated in the previous section) also receive 1 parameter ```retry_state```:
- ```my_stop(retry_state)```: parsed to ```stop``` and must return ```bool``` (wheter to stop or not);
- ```my_wait(retry_state)```: parsed to ```wait``` and must return ```float``` (number of seconds to wait before next retry);
- ```my_retry(retry_state)```: parsed to ```retry``` and must return ```bool``` (wheter to retry or not);
- ```my_before(retry_state)```: parsed to ```before```;
- ```my_before_sleep(retry_state)```: parsed to ```before_sleep```;
- ```my_after(retry_state)```: parsed to ```after```;

### Argument Behavior in Real Time ###
Another interesting feature is to change, in real time, an existing ```@retry``` decorator. This can be achieved by 2 different ways:
- via the ```retry_with()``` method (available through that same decorator and with the same keyword arguments as the original decorator);
- via the ```Retrying()``` class instance (assigned to a variable that later receives the function to be wrapped);

### Documentation ###
- ```tenacity``` Documentation - [Link](https://tenacity.readthedocs.io/en/latest/index.html?highlight=retry_error_callback#)
- ```tenacity``` Repo - [Link](https://github.com/jd/tenacity/tree/310058274ed22a345e9c3917c97b4afd6363d5a5/tenacity)

### Importing Dependencies ###

In [105]:
import random
import threading

import tenacity
from tenacity import stop, wait, retry
from tenacity import retry_if_result, retry_if_exception, retry_if_exception_type, retry_if_exception_message, retry_any
from tenacity import Retrying

### Main ###

**stop**

In [8]:
# Creating the decorator
@retry(stop= stop.stop_after_delay(max_delay=1)) # 'max_delay': in secs
def stop_1():
    print("Retries for 1 second") 
    raise(Exception) # Simulates an error
# Calling the function
stop_1()

Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1 second
Retries for 1

RetryError: RetryError[<Future at 0x10d24f6a0 state=finished raised Exception>]

In [9]:
# Creating the decorator
@retry(stop= stop.stop_after_attempt(max_attempt_number=3))
def stop_2():
    print("Retries for 3 attemps") 
    raise(Exception) # Simulates an error
# Calling the function
stop_2()

Retries for 3 attemps
Retries for 3 attemps
Retries for 3 attemps


RetryError: RetryError[<Future at 0x10afbb430 state=finished raised Exception>]

In [17]:
# Creating a threading Event
event_stop = threading.Event()
# Creating the decorator
@retry(stop= stop.stop_when_event_set(event=event_stop))
def stop_3():
    print("Retries until event is set randomly") 
    # Randomly setting the event
    if random.uniform(0,1) >= 0.75:
        print("Event is set")
        event_stop.set()
    raise(Exception) # Simulates an error
# Calling the function
stop_3()

Retries until event is set randomly
Retries until event is set randomly
Event is set


RetryError: RetryError[<Future at 0x10d1ea760 state=finished raised Exception>]

In [20]:
# Creating the decorator
@retry(stop= stop.stop_any(stop.stop_after_delay(max_delay=1),
                           stop.stop_after_attempt(max_attempt_number=3))) # Same as: stop= (stop.stop_after_delay(max_delay=1) | stop.stop_after_attempt(max_attempt_number=3))
def stop_4():
    print("Retries up to 3 attemps or 1s, whatever is reached first") 
    raise(Exception) # Simulates an error
# Calling the function
stop_4()

Retries up to 3 attemps or 1s, whatever is reached first
Retries up to 3 attemps or 1s, whatever is reached first
Retries up to 3 attemps or 1s, whatever is reached first


RetryError: RetryError[<Future at 0x10d2f69a0 state=finished raised Exception>]

In [21]:
# Creating the decorator
@retry(stop= stop.stop_all(stop.stop_after_delay(max_delay=1),
                           stop.stop_after_attempt(max_attempt_number=3))) # Same as: stop= (stop.stop_after_delay(max_delay=1) & stop.stop_after_attempt(max_attempt_number=3))
def stop_5():
    print("Retries up to 3 attemps and 1s, whatever is reached last") 
    raise(Exception) # Simulates an error
# Calling the function
stop_5()

Retries up to 3 attemps and 1s, whatever is reached last
Retries up to 3 attemps and 1s, whatever is reached last
Retries up to 3 attemps and 1s, whatever is reached last
Retries up to 3 attemps and 1s, whatever is reached last
Retries up to 3 attemps and 1s, whatever is reached last
Retries up to 3 attemps and 1s, whatever is reached last
Retries up to 3 attemps and 1s, whatever is reached last
Retries up to 3 attemps and 1s, whatever is reached last
Retries up to 3 attemps and 1s, whatever is reached last
Retries up to 3 attemps and 1s, whatever is reached last
Retries up to 3 attemps and 1s, whatever is reached last
Retries up to 3 attemps and 1s, whatever is reached last
Retries up to 3 attemps and 1s, whatever is reached last
Retries up to 3 attemps and 1s, whatever is reached last
Retries up to 3 attemps and 1s, whatever is reached last
Retries up to 3 attemps and 1s, whatever is reached last
Retries up to 3 attemps and 1s, whatever is reached last
Retries up to 3 attemps and 1s,

RetryError: RetryError[<Future at 0x10d1f6c70 state=finished raised Exception>]

**wait**

In [24]:
# Creating the decorator
@retry(stop= stop.stop_after_attempt(max_attempt_number=3), 
       wait= wait.wait_none())
def wait_1():
    print("Retries up to 3 attemps, not waiting between retries") 
    raise(Exception) # Simulates an error
# Calling the function
wait_1()

Retries up to 3 attemps, not waiting between retries
Retries up to 3 attemps, not waiting between retries
Retries up to 3 attemps, not waiting between retries


RetryError: RetryError[<Future at 0x10d4c90a0 state=finished raised Exception>]

In [25]:
# Creating the decorator
@retry(stop= stop.stop_after_attempt(max_attempt_number=3), 
       wait= wait.wait_fixed(wait=0.5))
def wait_2():
    print("Retries up to 3 attemps, waiting 0.5s between retries") 
    raise(Exception) # Simulates an error
# Calling the function
wait_2()

Retries up to 3 attemps, waiting 0.5s between retries
Retries up to 3 attemps, waiting 0.5s between retries
Retries up to 3 attemps, waiting 0.5s between retries


RetryError: RetryError[<Future at 0x10d3bc580 state=finished raised Exception>]

In [26]:
# Creating the decorator
@retry(stop= stop.stop_after_attempt(max_attempt_number=3), 
       wait= wait.wait_random(min=0.1, max=0.5))
def wait_3():
    print("Retries up to 3 attemps, waiting randomly (0.1s to 0.5s) between retries") 
    raise(Exception) # Simulates an error
# Calling the function
wait_3()

Retries up to 3 attemps, waiting randomly (0.1s to 0.5s) between retries
Retries up to 3 attemps, waiting randomly (0.1s to 0.5s) between retries
Retries up to 3 attemps, waiting randomly (0.1s to 0.5s) between retries


RetryError: RetryError[<Future at 0x10d4e7a00 state=finished raised Exception>]

In [27]:
# Creating the decorator
@retry(stop= stop.stop_after_attempt(max_attempt_number=3), 
       wait= wait.wait_incrementing(start=0, increment=0.25, max=1))
def wait_4():
    print("Retries up to 3 attemps, waiting with 0.25s increments between retries") 
    raise(Exception) # Simulates an error
# Calling the function
wait_4()

Retries up to 3 attemps, waiting with 0.25s increments between retries
Retries up to 3 attemps, waiting with 0.25s increments between retries
Retries up to 3 attemps, waiting with 0.25s increments between retries


RetryError: RetryError[<Future at 0x10d1b5730 state=finished raised Exception>]

In [28]:
# Creating the decorator
@retry(stop= stop.stop_after_attempt(max_attempt_number=3), 
       wait= wait.wait_exponential(multiplier=0.5, exp_base=2, min=0, max=2))
def wait_5():
    print("Retries up to 3 attemps, waiting exponentially (w/o jitter) between retries") 
    raise(Exception) # Simulates an error
# Calling the function
wait_5()

Retries up to 3 attemps, waiting exponentially (w/o jitter) between retries
Retries up to 3 attemps, waiting exponentially (w/o jitter) between retries
Retries up to 3 attemps, waiting exponentially (w/o jitter) between retries


RetryError: RetryError[<Future at 0x10d4c9b80 state=finished raised Exception>]

In [29]:
# Creating the decorator
@retry(stop= stop.stop_after_attempt(max_attempt_number=3), 
       wait= wait.wait_exponential_jitter(initial=1, exp_base=2, max=2, jitter=0.5))
def wait_6():
    print("Retries up to 3 attemps, waiting exponentially (w/ jitter) between retries") 
    raise(Exception) # Simulates an error
# Calling the function
wait_6()

Retries up to 3 attemps, waiting exponentially (w/ jitter) between retries
Retries up to 3 attemps, waiting exponentially (w/ jitter) between retries
Retries up to 3 attemps, waiting exponentially (w/ jitter) between retries


RetryError: RetryError[<Future at 0x10d44d250 state=finished raised Exception>]

In [30]:
# Creating the decorator
@retry(stop= stop.stop_after_attempt(max_attempt_number=3), 
       wait= wait.wait_random_exponential(multiplier=0.5, exp_base=2, min=0, max=2))
def wait_7():
    print("Retries up to 3 attemps, waiting exponentially random between retries") 
    raise(Exception) # Simulates an error
# Calling the function
wait_7()

Retries up to 3 attemps, waiting exponentially random between retries
Retries up to 3 attemps, waiting exponentially random between retries
Retries up to 3 attemps, waiting exponentially random between retries


RetryError: RetryError[<Future at 0x10d53a910 state=finished raised Exception>]

In [41]:
# Creating the decorator
@retry(stop= stop.stop_after_attempt(max_attempt_number=3), 
       wait= wait.wait_chain(*[wait.wait_fixed(0.25) for i in range(2)] + 
                              [wait.wait_fixed(0.5) for j in range(1)]))
def wait_8():
    print("Retries up to 3 attemps, waiting 0.25s for first 2 attempts and 0.5s for the 3rd") 
    raise(Exception) # Simulates an error
# Calling the function
wait_8()

Retries up to 3 attemps, waiting 0.25s for first 2 attempts and 0.5s for the 3rd
Retries up to 3 attemps, waiting 0.25s for first 2 attempts and 0.5s for the 3rd
Retries up to 3 attemps, waiting 0.25s for first 2 attempts and 0.5s for the 3rd


RetryError: RetryError[<Future at 0x10d6e3d30 state=finished raised Exception>]

**retry**

In [54]:
# Creating an evaluator function (returns bool)
def is_1(value):
    return value == 1 # [bool]

# Creating the decorator
@retry(stop= stop.stop_after_attempt(max_attempt_number=3), 
       wait= wait.wait_fixed(wait=0.5),
       retry= retry_if_result(predicate= is_1)) # 'retry_if_not_result' also available
def retry_1():
    print("Retries if return is 1") 
    # Randomly setting the return
    if random.uniform(0,1) >= 0.25:
        print("Returned 1")
        return 1
    else:
        print("Returned 0")
        return 0
# Calling the function
retry_1()

Retries if return is 1
Returned 1


Retries if return is 1
Returned 0


0

In [76]:
# Creating a custom exception
class BrunoException(Exception):
    def __init__(self):
        return None
    
# Creating an evaluator function (returns bool)
def is_bruno_exception(exception):
    return isinstance(exception, BrunoException)

# Creating the decorator
@retry(stop= stop.stop_after_attempt(max_attempt_number=3), 
       wait= wait.wait_fixed(wait=0.5),
       retry= retry_if_exception(predicate= is_bruno_exception))
def retry_2():
    print("Retries if raised exception is evaluated as 'BrunoException'") 
    # Randomly setting the return
    if random.uniform(0,1) >= 0.25:
        print("Raised BrunoException")
        raise(BrunoException)
    else:
        print("Raised Exception")
        raise(Exception)
# Calling the function
retry_2()

Retries if raised exception is evaluated as 'BrunoException'
Raised BrunoException
Retries if raised exception is evaluated as 'BrunoException'
Raised Exception


Exception: 

In [64]:
# Creating a custom exception
class BrunoException(Exception):
    def __init__(self):
        return None

# Creating the decorator
@retry(stop= stop.stop_after_attempt(max_attempt_number=3), 
       wait= wait.wait_fixed(wait=0.5),
       retry= retry_if_exception_type(exception_types= BrunoException)) # 'retry_if_not_exception_type' and 'retry_unless_exception_type' also available
                                                                        # If multiple 'exception_types', encapsulate with tuple
def retry_3():
    print("Retries if raised exception is of type 'BrunoException'") 
    # Randomly setting the raise
    if random.uniform(0,1) >= 0.25:
        print("Raised BrunoException")
        raise(BrunoException)
    else:
        print("Raised Exception")
        raise(Exception)
# Calling the function
retry_3()

Retries if raised exception is of type 'BrunoException'
Raised BrunoException


Retries if raised exception is of type 'BrunoException'
Raised BrunoException
Retries if raised exception is of type 'BrunoException'
Raised Exception


Exception: 

In [70]:
# Creating the decorator
@retry(stop= stop.stop_after_attempt(max_attempt_number=3), 
       wait= wait.wait_fixed(wait=0.5),
       retry= retry_if_exception_message(message= "This is a BrunoException")) # 'retry_if_not_exception_message' also available
def retry_4():
    print("Retries if raised exception has 'This is a BrunoException' message") 
    # Randomly setting the raise
    if random.uniform(0,1) >= 0.25:
        print("Raised 'This is a BrunoException' message")
        raise(Exception("This is a BrunoException"))
    else:
        print("Raised 'This is a generic Exception' message")
        raise(Exception("This is a generic Exception"))
# Calling the function
retry_4()

Retries if raised exception has 'This is a BrunoException' message
Raised 'This is a BrunoException' message
Retries if raised exception has 'This is a BrunoException' message
Raised 'This is a generic Exception' message


Exception: This is a generic Exception

In [87]:
# Creating the decorator
@retry(stop= stop.stop_after_attempt(max_attempt_number=3), 
       wait= wait.wait_fixed(wait=0.5),
       retry= retry_any(retry_if_exception_message(message= "This is a BrunoException"),
                        retry_if_exception_message(message= "This is a MontoniException"))) # 'retry_all' also available
def retry_5():
    print("Retries if raised exception has 'This is a BrunoException' or 'This is a MontoniException' message") 
    # Randomly setting the raise
    if random.uniform(0,1) >= 0.25:
        print("Raised 'This is a BrunoException' message")
        raise(Exception("This is a BrunoException"))
    else:
        print("Raised 'This is a generic Exception' message")
        raise(Exception("This is a generic Exception"))
# Calling the function
retry_5()

Retries if raised exception has 'This is a BrunoException' or 'This is a MontoniException' message
Raised 'This is a BrunoException' message
Retries if raised exception has 'This is a BrunoException' or 'This is a MontoniException' message
Raised 'This is a BrunoException' message
Retries if raised exception has 'This is a BrunoException' or 'This is a MontoniException' message
Raised 'This is a BrunoException' message


RetryError: RetryError[<Future at 0x10d499160 state=finished raised Exception>]

**reraise**

In [94]:
# Creating the decorator
@retry(stop= stop.stop_after_attempt(max_attempt_number=3), 
       reraise= True)
def reraise_1():
    print("Retries up to 3 attemps, reraising the original exception") 
    raise(Exception("This is the original exception")) # Simulates an error
# Calling the function
reraise_1()

Retries up to 3 attemps, reraising the original exception
Retries up to 3 attemps, reraising the original exception
Retries up to 3 attemps, reraising the original exception


Exception: This is the original exception

**statistics** (attribute)

In [97]:
# Creating the decorator + function to raise exception
@retry(stop= stop.stop_after_attempt(max_attempt_number=3),
       wait= wait.wait_fixed(wait=0.5))
def raise_exception(value):
    raise(Exception)

# Triggering the exception
try:
    raise_exception()
except Exception: # So we don't raise an exception
    pass 
# Accessing the statistics of the last retry
raise_exception.retry.statistics

{'start_time': 12812.282753902,
 'attempt_number': 3,
 'idle_for': 1.0,
 'delay_since_first_attempt': 1.0031160329999693}

**custom_callbacks**

In [123]:
# Creating a callback function
def bruno_callback(retry_state):
    print("Attempt Number: {}".format(retry_state.attempt_number)) # Attempt number
    print("Attempt Start Time: {}".format(retry_state.start_time)) # Attempt start timestamp
    print("Sleep Time: {}".format(retry_state.idle_for)) # Time spent sleeping in retries
    print("Wrapped Function: {}".format(retry_state.fn)) # Function wrapped by attempt
    print("args Function: {}".format(retry_state.args)) # args wrapped by attempt
    print("kwargs Function: {}".format(retry_state.kwargs)) # kwargs wrapped by attempt
    print("Outcome: {}".format(retry_state.outcome)) # kwargs wrapped by attempt

# Creating a 'Retrying' instance
retrying_1 = Retrying(stop= stop.stop_after_attempt(max_attempt_number=3),
                      retry_error_callback= bruno_callback)
# Creating a dummy function
def callback_1():
    print("Retries with custom callback") 
    raise(Exception) # Simulates an error
# Calling the function
retrying_1(callback_1)

Retries with custom callback
Retries with custom callback
Retries with custom callback
Attempt Number: 3
Attempt Start Time: 15342.351313147
Sleep Time: 0.0
Wrapped Function: <function callback_1 at 0x10d43a940>
args Function: ()
kwargs Function: {}
Outcome: <Future at 0x10d3beb80 state=finished raised Exception>


---